<a href="https://colab.research.google.com/github/lucaschenu/Archivo-materia-IA/blob/main/CWEEKLY_REPORT_LUCAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IMPORTAR BIBLIOTECAS
!pip install optuna
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 19.0 MB/s eta 0:00:00


In [ ]:
#CARREGAR ARCHIVO
from google.colab import files
uploaded = files.upload()

Saving 01-04T184148_000_mode1.csv to 01-04T184148_000_mode1.csv


In [ ]:
print(df.columns)

Index(['timestamp', 'pCut::Motor_Torque',
       'pCut::CTRL_Position_controller::Lag_error',
       'pCut::CTRL_Position_controller::Actual_position',
       'pCut::CTRL_Position_controller::Actual_speed',
       'pSvolFilm::CTRL_Position_controller::Actual_position',
       'pSvolFilm::CTRL_Position_controller::Actual_speed',
       'pSvolFilm::CTRL_Position_controller::Lag_error',
       'pSpintor::VAX_speed'],
      dtype='object')


In [ ]:
# FUNÇÃO DE MÉTRICAS
def metricas(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

In [ ]:
 #FUNÇÃO OBJETIVO (Optuna)

def criar_modelo(trial):

    n_camadas = trial.suggest_int("n_camadas", 1, 5)
    n_neuronios = trial.suggest_int("n_neuronios", 32, 256)
    dropout_rate = trial.suggest_float("dropout", 0.0, 0.5)

    ativacao = trial.suggest_categorical(
        "ativacao", ["relu", "tanh", "elu"]
    )

    learning_rate = trial.suggest_float(
        "lr", 1e-5, 1e-2, log=True
    )

    modelo = keras.Sequential()

    for _ in range(n_camadas):
        modelo.add(layers.Dense(n_neuronios, activation=ativacao))
        modelo.add(layers.Dropout(dropout_rate))

    modelo.add(layers.Dense(1))  # saída

    otimizador = keras.optimizers.Adam(learning_rate=learning_rate)

    modelo.compile(
        loss="mse",
        optimizer=otimizador,
        metrics=["mae"]
    )

    return modelo


def objetivo(trial):
    modelo = criar_modelo(trial)

    early = keras.callbacks.EarlyStopping(
        patience=20, restore_best_weights=True
    )

    modelo.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=trial.suggest_int("batch_size", 16, 128),
        callbacks=[early],
        verbose=0
    )

    pred = modelo.predict(X_test).flatten()
    rmse, _, _ = metricas(y_test, pred)

    return rmse

In [ ]:
# RODAR OPTUNA
study = optuna.create_study(direction="minimize")
study.optimize(objetivo, n_trials=40)

print("Melhores hiperparâmetros:", study.best_params)


[I 2025-11-30 03:56:23,041] A new study created in memory with name: no-name-ef91904c-f587-4e29-8bbd-d327946c6e2e


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


[I 2025-11-30 03:56:29,095] Trial 0 finished with value: 7330.4035585788 and parameters: {'n_camadas': 1, 'n_neuronios': 253, 'dropout': 0.2574175464171564, 'ativacao': 'relu', 'lr': 0.00015264262213275694, 'batch_size': 125}. Best is trial 0 with value: 7330.4035585788.


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


[I 2025-11-30 03:56:37,285] Trial 1 finished with value: 12103.30666452562 and parameters: {'n_camadas': 2, 'n_neuronios': 114, 'dropout': 0.27764969588804017, 'ativacao': 'relu', 'lr': 0.0001867916337795491, 'batch_size': 46}. Best is trial 0 with value: 7330.4035585788.


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


[I 2025-11-30 04:04:00,137] Trial 38 finished with value: 99.16132381922333 and parameters: {'n_camadas': 3, 'n_neuronios': 203, 'dropout': 0.26687442217572016, 'ativacao': 'relu', 'lr': 0.0039990019156662705, 'batch_size': 35}. Best is trial 12 with value: 0.5377274681678337.


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


[I 2025-11-30 04:04:15,687] Trial 39 finished with value: 0.5377397267655049 and parameters: {'n_camadas': 4, 'n_neuronios': 189, 'dropout': 0.2965405611658751, 'ativacao': 'tanh', 'lr': 0.0016197449765210934, 'batch_size': 43}. Best is trial 12 with value: 0.5377274681678337.


Melhores hiperparâmetros: {'n_camadas': 5, 'n_neuronios': 140, 'dropout': 0.3957598298656548, 'ativacao': 'tanh', 'lr': 0.009240587870945323, 'batch_size': 64}


In [ ]:
df = pd.read_csv("01-04T184148_000_mode1.csv")  # substitua pelo seu dataset
TARGET_COLUMN = 'pCut::Motor_Torque'
X = df.drop(TARGET_COLUMN, axis=1).values
y = df[TARGET_COLUMN].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# TREINAR O MELHOR MODELO
best_model = criar_modelo(study.best_trial)

early = keras.callbacks.EarlyStopping(
    patience=20,
    restore_best_weights=True
)

best_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=300,
    batch_size=study.best_params["batch_size"],
    callbacks=[early],
    verbose=1
)

y_pred = best_model.predict(X_test).flatten()

rmse, mae, r2 = metricas(y_test, y_pred)
print("RMSE:", rmse)
print("MAE :", mae)
print("R2  :", r2)

Please review the output above and let me know which column you intend to use as the `target` variable. Once you provide the correct column name, I can update the code to fix the error.